# **Análise de Dados**
### <font color='red'> *Análise de dados do Censo Agropecuário do Brasil de 1985 - 2017*
*by [Matheus Fellipe do Carmo Barros](https://linktr.ee/matheusfcbarros)*

---


### <font color=blue> Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import basedosdados as bd
import matplotlib.pyplot as plt
import plotly as py
import plotly.express as px
import plotly.graph_objects as go

### <font color=blue> 1. Carregando a base de dados

> Utilizei o pacote ``basedosdados`` para fazer a consulta SQL na BD+ (Big Query GCP)

Usei a base Censo Agropecuário (br_ibge_censo_agropecuario) para realizar as análises e realizei um cruzamento com a base 
(br_bd_diretorios_brasil) para poder ter também os nomes dos municípios. Ambas as bases estão disponíveis no site da [Base dos Dados](https://basedosdados.org/dataset?order_by=score)


> Query SQL, para download da base, em uma forma mais simples

In [ ]:
base_censo1 = bd.read_sql(
    '''
    SELECT censo.*, municipio.nome
        FROM basedosdados.br_ibge_censo_agropecuario.municipio AS censo 
        INNER JOIN basedosdados.br_bd_diretorios_brasil.municipio AS municipio
            ON censo.id_municipio = municipio.id_municipio
    ''',
    billing_project_id='teste1-360117'
)

> Query SQL, para download da base, usando subqueries

> 
base_censo2 = bd.read_sql(

    '''
    SELECT censo.*, nome_municipio.nome

        FROM basedosdados.br_ibge_censo_agropecuario.municipio AS censo 

        INNER JOIN (SELECT municipio.id_municipio, municipio.nome

                        FROM basedosdados.br_bd_diretorios_brasil.municipio AS municipio) AS nome_municipio

            ON censo.id_municipio = nome_municipio.id_municipio
    ''',

    billing_project_id='teste1-360117'
    
 )

> Usando ``read_table``

> 
#Teste usando read_table

    df = bd.read_table(dataset_id='br_ibge_censo_agropecuario',

    table_id='municipio',

    billing_project_id="teste1-360117")

Como forma de teste fiz o download da base de dados ``br_ibge_censo_agropecuario`` acrescida da informação dos nomes dos municípios e utlizando ``read_sql`` com uma query mais simples e ele foi realizado em 30s. Já o download da mesma base de dados utilizando subqueries foi realizado em 37s. O download da base ``br_ibge_censo_agropecuario`` , sem a informação dos nomes dos municípios , utilizando ``read_table`` gastou 49.8s.

### <font color=blue> 2. Informações da base de dados

In [ ]:
base_censo1

In [ ]:
#Tipos presente na base
base_censo1.dtypes

In [ ]:
base_censo1.info()

### <font color=blue> 3. Tratando a base de dados

In [ ]:
#convertendo id_municipio em tipo numerico para poder realizar uma melhor verificação de valores vazios
base_censo1["id_municipio"] = pd.to_numeric(base_censo1["id_municipio"], errors = "coerce")
base_censo1

In [ ]:
#Tipos presente na base após conversão da id_municipio
base_censo1.dtypes

In [ ]:
#Reordenando a base para uma melhor vizualização
col_nome = base_censo1["nome"]
base_censo1_ordenada = base_censo1
if 'nome_municipio' not in base_censo1_ordenada:
    base_censo1_ordenada.insert(3,'nome_municipio',col_nome)
    


In [ ]:
#Base reordenada
base_censo1_ordenada

In [ ]:
#Verificando valores vazios para saber se posso eliminar alguma coluna
contar_Valores_NaN = list(base_censo1_ordenada.isnull().sum().sort_values(ascending = False))

contar_Valores_NaN 



In [ ]:
#Eliminando a coluna nome (Ja reordenei ela para outra posição) e eliminando a coluna (proporcao_despesa_salarios) ja que ela nao tem nenhum valor
if ("proporcao_despesa_salarios" or "nome") in base_censo1_ordenada:
        base_censo1_ordenada = base_censo1_ordenada.drop(columns=["proporcao_despesa_salarios","nome"])
     
base_censo1_ordenada


#### Valores vazios

        
        Resolvi tratar os valores vazios preenchendo os campos com a mediana da coluna 

In [ ]:
#Função que preenche os campos vazios
def preenche_campos_vazios (baseDados):
    nome_colunas = list(baseDados.columns.values.tolist())
    num_col = len(nome_colunas)
    controle = 0
    for i in nome_colunas:
        contar_Valores_Vazios = baseDados[nome_colunas[controle]].isnull().sum()
        #print (contar_Valores_Vazios)
        if contar_Valores_Vazios > 0:
            #print (contar_Valores_Vazios)
            #print(i)
            mediana = baseDados[nome_colunas[controle]].median()
            mediana = int(mediana)
            baseDados[nome_colunas[controle]] = baseDados[nome_colunas[controle]].fillna(mediana)
        controle = controle + 1 
    return baseDados

In [ ]:
#Chamando a função que preenche os campos vazios
preenche_campos_vazios(base_censo1_ordenada)

### <font color=blue> 4. Preparação dos dados para visualização

> ``Dataframes auxiliares``

Criei esses dataframes auxiliares para agrupar os dados e facilitar a visualização

### Bovinos

In [ ]:
#Área utilizada por ano
total_area_pastagem = base_censo1_ordenada.groupby("ano").area_pastagem.sum()
total_area_pastagem = pd.DataFrame(total_area_pastagem)
total_area_pastagem = total_area_pastagem.reset_index()
total_area_pastagem

In [ ]:
#Cálculo da quantidade de bovinos por ano
total_quantidade_bovinos = base_censo1_ordenada.groupby("ano").quantidade_bovinos_total.sum()
total_quantidade_bovinos = pd.DataFrame(total_quantidade_bovinos)
total_quantidade_bovinos = total_quantidade_bovinos.reset_index()
total_quantidade_bovinos

In [ ]:
#Cálculo do valor da produção de bovinos por ano
valor_producao_total_animal = base_censo1_ordenada.groupby("ano").valor_producao_total_animal.sum()
valor_producao_total_animal = pd.DataFrame(valor_producao_total_animal)
valor_producao_total_animal = valor_producao_total_animal.reset_index()
valor_producao_total_animal



### Arroz

In [ ]:
#Área utilizada por ano
total_area_arroz = base_censo1_ordenada.groupby("ano").area_arroz.sum()
total_area_arroz = pd.DataFrame(total_area_arroz)
total_area_arroz = total_area_arroz.reset_index()
total_area_arroz


In [ ]:
#Cálculo da produção de arroz, em toneladas, por ano
total_producao_arroz = base_censo1_ordenada.groupby("ano").producao_total_arroz.sum()
total_producao_arroz = pd.DataFrame(total_producao_arroz)
total_producao_arroz = total_producao_arroz.reset_index()
total_producao_arroz

In [ ]:
#Cálculo do valor da produção de arroz por ano
valor_total_arroz = base_censo1_ordenada.groupby("ano").valor_total_arroz.sum()
valor_total_arroz = pd.DataFrame(valor_total_arroz)
valor_total_arroz = valor_total_arroz.reset_index()
valor_total_arroz

### Soja

In [ ]:
#Área utilizada no período
total_area_soja= base_censo1_ordenada.groupby("ano").area_soja.sum()
total_area_soja = total_area_soja.reset_index()
total_area_soja

In [ ]:
#Cálculo da produção de soja, em toneladas, no período 
total_producao_soja = base_censo1_ordenada.groupby("ano").producao_total_soja.sum()
total_producao_soja = total_producao_soja.reset_index()
total_producao_soja

In [ ]:
#Calculo do valor da produção de soja no período
valor_total_soja = base_censo1_ordenada.groupby("ano").valor_total_soja.sum()
valor_total_soja = valor_total_soja.reset_index()
valor_total_soja

In [ ]:
#Função que faz o merge entre os 3 dataframes auxiliares de cada produto
def merge_df_aux(df1,df2,df3):
    df_merge = pd.merge(df1, df2,how= 'outer' )
    df_merge = pd.merge(df_merge, df3, how= 'outer')
    return df_merge


### <font color=blue> 5. Gerando Gráficos

### Bovinos

In [ ]:
# Preparando variaveis para criação do gráfico
ano = list(merge_df_aux(total_area_pastagem,total_quantidade_bovinos,valor_producao_total_animal).ano)
area_Bovinos  = list(merge_df_aux(total_area_pastagem,total_quantidade_bovinos,valor_producao_total_animal).area_pastagem)
qtd_Bovinos = list(merge_df_aux(total_area_pastagem,total_quantidade_bovinos,valor_producao_total_animal).quantidade_bovinos_total)
vlr_Bovinos = list(merge_df_aux(total_area_pastagem,total_quantidade_bovinos,valor_producao_total_animal).valor_producao_total_animal)

In [ ]:
#Gerando o gráfico
graf_Bovinos = go.Figure(data=[

    go.Bar(name='Área (m2)', x=ano, y=area_Bovinos ),

    go.Bar(name='Quantidade',x=ano, y=qtd_Bovinos ),

    go.Bar(name='Valor (x100k)',x=ano, y= vlr_Bovinos ) 
])


graf_Bovinos.update_layout(title='Bovinos', xaxis_title='Ano', yaxis_title='Bovinos (Área, Quantidade, Valor)',
                            template = 'plotly_white', title_x=0.5, barmode='group',  autosize=False,
    width=600, height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4)
)
py.offline.plot(graf_Bovinos, filename = "Grafico_Bovinos.html")
graf_Bovinos.show()

### Arroz

In [ ]:
# Preparando variaveis para criação do gráfico
ano = list(merge_df_aux(total_area_arroz,total_producao_arroz,valor_total_arroz).ano)
area_Arroz  = list(merge_df_aux(total_area_arroz,total_producao_arroz,valor_total_arroz).area_arroz)
qtd_Arroz = list(merge_df_aux(total_area_arroz,total_producao_arroz,valor_total_arroz).producao_total_arroz)
vlr_Arroz = list(merge_df_aux(total_area_arroz,total_producao_arroz,valor_total_arroz).valor_total_arroz)

In [ ]:
#Gerando o gráfico
graf_Arroz = go.Figure(data=[

    go.Bar(name='Área (m2)', x=ano, y=area_Arroz ),

    go.Bar(name='Quantidade (Toneladas)',x=ano, y=qtd_Arroz ),

    go.Bar(name='Valor (x100k)',x=ano, y= vlr_Arroz ) 
])


graf_Arroz.update_layout(title='Arroz', xaxis_title='Ano', yaxis_title='Arroz (Área, Quantidade, Valor)',
                            template = 'plotly_white', title_x=0.4, barmode='group',  autosize=False,
    width=600,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4)
)
py.offline.plot(graf_Arroz, filename = "Grafico_Arroz.html")
graf_Arroz.show()

### Soja

In [ ]:
ano = list(merge_df_aux(total_area_soja,total_producao_soja,valor_total_soja).ano)
area_Soja  = list(merge_df_aux(total_area_soja,total_producao_soja,valor_total_soja).area_soja)
qtd_Soja = list(merge_df_aux(total_area_soja,total_producao_soja,valor_total_soja).producao_total_soja)
vlr_Soja = list(merge_df_aux(total_area_soja,total_producao_soja,valor_total_soja).valor_total_soja)

In [ ]:
#Gerando o gráfico
graf_Soja = go.Figure(data=[

    go.Bar(name='Área (m2)', x=ano, y=area_Soja ),

    go.Bar(name='Quantidade (Toneladas)',x=ano, y=qtd_Soja ),

    go.Bar(name='Valor (x100k)',x=ano, y= vlr_Soja ) 
])


graf_Soja.update_layout(title='Soja', xaxis_title='Ano', yaxis_title='Soja (Área, Quantidade, Valor)',
                            template = 'plotly_white', title_x=0.4, barmode='group',  autosize=False,
    width=600,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4)
)
py.offline.plot(graf_Soja, filename = "Grafico_Soja.html")
graf_Soja.show()